In [1]:
## flash attention是一个算法。
## 大概的意思就是，能够不要读取一整个QKV矩阵，还能够开展attention计算。
## 我擦嘞。

## 额外参考了：
### https://zhuanlan.zhihu.com/p/676655352

In [1]:
import torch
import torch.nn as nn
import math

In [113]:
N = 80 
d = 32
M = 1000 ## SRAM 大小为M

In [114]:
q_linear = nn.Linear(d, d)
k_linear = nn.Linear(d, d)
v_linear = nn.Linear(d, d)

In [115]:
x = nn.Embedding(N, d).weight ## 这个就是咱们姑且这么设定吧。
q = q_linear(x)
k = k_linear(x)
v = v_linear(x)

In [116]:
Bc = math.ceil(
    M/(4*d)
) ## 设置块大小，每块大小为这么大。。。
Br = min(
    math.ceil(
        M/(4*d)
    ),
    d
) ## 设置块大小，每块大小为这么大。。。
Bc, Br

(8, 8)

In [117]:
## 在HBM中初始化：
o = torch.zeros(N, d)
l = torch.zeros(N)
m = torch.tensor([float('-inf')] * N, dtype=torch.float32) 

In [118]:
Tr = math.ceil(N/Br)
Tc = math.ceil(N/Bc)
Tr, Tc ## 将矩阵分成这么多块。

(10, 10)

In [119]:
q_blocks = torch.split(q, Br, dim=0) ## 这个是新学到的方法，也就是将原来的矩阵分块。
o_blocks = torch.split(o, Br, dim=0) ## 这个是新学到的方法，也就是将原来的矩阵分块。
k_blocks = torch.split(k, Bc, dim=0) ## 这个是新学到的方法，也就是将原来的矩阵分块。
v_blocks = torch.split(v, Bc, dim=0) ## 这个是新学到的方法，也就是将原来的矩阵分块。

l_blocks = torch.split(l, Br, dim=0)
m_blocks = torch.split(m, Br, dim=0)

In [125]:
new_o_blocks, new_l_blocks, new_m_blocks = [None] * len(o_blocks), [None] * len(l_blocks), [None] * len(m_blocks)

for j in range(Tc):
    Kj, Vj = k_blocks[j], v_blocks[j]
    for i in range(Tr):
        mi = m_blocks[i]
        li = l_blocks[i]
        Oi = o_blocks[i]
        
        Qi = q_blocks[i]
        Sij = torch.einsum('...id,...jd->...ij', Qi, Kj)
        mij, _ = Sij.max(axis=1, keepdims=True)
        Pij = torch.exp(Sij - mij)
        lij = Pij.sum(axis=1, keepdims=True)

        mi_new = torch.maximum(mi.unsqueeze(1), mij)
        li_new = torch.exp(mi.unsqueeze(1) - mi_new) * li.unsqueeze(1) + torch.exp(mij - mi_new) * lij

        new_o_blocks[i] = \
        (li.unsqueeze(1) / li_new) * torch.exp(mi.unsqueeze(1) - mi_new) * Oi\
        + (torch.exp(mij - mi_new) / li_new) * torch.mm(
            Pij, Vj
        )
        new_l_blocks[i] = li_new
        new_m_blocks[i] = mi_new
    # break

In [128]:
o = torch.cat(new_o_blocks, dim=0)
l = torch.cat(new_l_blocks, dim=0)
m = torch.cat(new_m_blocks, dim=0)

In [133]:
## 总结了一下：
### 对角矩阵的逆矩阵，实际上就是对角元素变成原来的倒数（0的话还是变成0）
### 算法有的时候没有实现对。
### 书本上的算法本身有的时候就是错的。。。这个感觉防不胜防。